In [ ]:
# NOTES
# For testing: should omit ratings for userid/gameid pairs from entire df
# do not omit entire rows
# try test/train split with df before changing to pivot_table
# ideally, if userid entries have the same freq, omit 10-15% of uniq userid entries
# does it make sense to try and keep user-similarity matrix as close as possible to training on full df?
# ...aka... does omitting certain test data drastically change user-similarity matrix?


# Consider cross-validation for more accurate testing

# Consider wrapping in custom sklearn estimator / classifier, maybe inherit from base classes

# README
# Datafield has 'user_ids' and their 'rating' (1-10) of various 'game_ids'
# Fields = userID , gameID , rating
# Prediction of unrated games for each user_id is made using:
# k-nearest neighbors found using cosine-similarity
# predicted rating based on those near-neighbors' rating weighted by cosine-similarity users

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('inputs/boardgame-elite-users.csv')

train, test = train_test_split(df, test_size=.15)

In [9]:
#print(train.userID.unique)
print(train.shape)
print(test.shape)
x = train.groupby('userID')
print(len(x.groups))
y = test.groupby('userID')
print(len(y.groups))
# this will work

(50399, 3)
(8894, 3)
199
199


In [3]:
rating_ptable = df.pivot_table(index='userID', columns='gameID', values='rating') 

rating_ptable = rating_ptable.fillna(0)

In [4]:
from sklearn.preprocessing import normalize

norm_train = normalize(train)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(norm_train)

In [9]:
nbor_indx = sim.argsort()[:,::-1]

# HELPER FUNCTIONS
def get_users_that_rated(gameid):
    return train[gameid].where(train[gameid]>0).dropna().index.tolist()
    
def get_k_nearest_neighbors(userid, gameid, k):
    assert(k>1)
    pot_nbors = get_users_that_rated(gameid)
    userid_indx = train.index.get_loc(userid)
    
    sim_userIDs = []
    for x in nbor_indx[userid_indx]:
        if train.index[x] in pot_nbors:
            sim_userIDs.append(train.index[x])
        if len(sim_userIDs)>k:
            break
    return sim_userIDs[1:]

def round_to_point5(someFloat):
    score = np.around(someFloat, decimals=1)
    first_digit = score // 1
    last_digit = (score % 1) * 10
    carry = 0
    if 0 <= last_digit <= 2:
        return float(first_digit)
    elif 2 < last_digit <= 7:
        return float(first_digit + .5)
    elif last_digit > 7:
        return float(first_digit + 1)
        
    

In [10]:
# Takes userID, gameID, optional k
# Returns predicted rating for gameID
def predict_rating(uid, gameid, k=10):
    sim_uids = get_k_nearest_neighbors(uid, gameid, k)
    accm_score = 0.0
    accm_weight = 0.0
    for nbor_id in sim_uids:
        sim_indx_x = train.index.get_loc(uid)
        sim_indx_y = train.index.get_loc(nbor_id)
        accm_score += (train.loc[nbor_id][gameid] * sim[sim_indx_x][sim_indx_y])
        accm_weight += sim[sim_indx_x][sim_indx_y]
    score = accm_score / accm_weight
    score = round_to_point5(score)
    return score

predict_rating(5480, 97903)

6.5

In [ ]:
# -Generate matrix of all predictions
# -ensure shape of test data is the same
# use sklearn.metrics accuracy_score as below

>>> import numpy as np
>>> from sklearn.metrics import accuracy_score
>>> y_pred = [0, 2, 1, 3]
>>> y_true = [0, 1, 2, 3]
>>> accuracy_score(y_true, y_pred)
0.5
>>> accuracy_score(y_true, y_pred, normalize=False)
2

In [12]:
print(test.values)

[[ 0.    0.    8.   ...  9.    8.    0.  ]
 [ 7.    6.75  0.   ...  0.    9.5   0.  ]
 [ 7.    6.5   8.   ...  0.    8.5   0.  ]
 ...
 [ 0.    7.    0.   ...  0.    0.    9.  ]
 [ 0.    7.    4.   ...  9.   10.    0.  ]
 [ 5.    7.75  5.   ...  0.    0.    0.  ]]


In [21]:
print(test.shape)
print(train.shape)
# are the same userids in both df? no, knn doesnt make sense to use then


(30, 402)
(169, 402)
Int64Index([ 55092,  99306, 123709, 121369,  93535, 127141, 180775,  32282,
            103398, 155243,  54974,  49788,  27106,  55101,   5217, 112817,
            156708, 189973,  83092,  95213,  77337,  14310, 160442,  58582,
            176216, 154394, 112355, 148941, 144682,  96412],
           dtype='int64', name='userID')
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in


In [19]:
user_ids = []
for row in test.itertuples():
    user_ids.append(row[0])
game_ids = []
for x in test.columns:
    game_ids.append(x)

pred_matrix = []
for user in user_ids:
    row = []
    for gid in game_ids:
        row.append(predict_rating(user,gid))
    pred_matrix.append(row)

KeyError: 55092